In [386]:
import requests
import pandas as pd
import pprint
import csv
from datetime import datetime
import logging
import os
import logging.handlers
# NISE
NISE = "nise gwal"
NISE_STATION = "NISE Gwal Pahari, Gurugram, India"
NISE_OUTPUT = "../../data/cron_job_data/nise_cron_output"

# Sector 51
SECTOR_51 = "Sector-51, Gurugram"
SECTOR_51_STATION = "Sector-51, Gurugram, India"
SECTOR_51_OUTPUT = "../../data/cron_job_data/sector_51_cron_output"

# Teri gram
TERI_GRAM = "Teri Gram"
TERI_GRAM_STATION = "Teri Gram, Gurugram"
TERI_GRAM_OUTPUT = "../../data/cron_job_data/teri_gram_cron_output"

# Vikas Sadan
VIKAS_SADAN = "Vikas Sadan"
VIKAS_SADAN_STATION = "Vikas Sadan Gurgaon"
VIKAS_SADAN_OUTPUT = "../../data/cron_job_data/vikas_sadan_cron_output"

stations = [(NISE,  NISE_OUTPUT), (SECTOR_51, SECTOR_51_OUTPUT), (TERI_GRAM, TERI_GRAM_OUTPUT), (VIKAS_SADAN, VIKAS_SADAN_OUTPUT)]

def get_api_token():
    try:
        return os.environ['API_TOKEN']
    except KeyError:
        print("KeyError")
         # If running locally, use an alternative method to get the API token
        return input("Enter your API token: ")
    
# Incase if api fails, write to file with previous day
# In 2nd cron daily job, Incase if api fails, write to file with previous day
def setData(station, output_file):
    try:
        # Logging
        logger = logging.getLogger(__name__)
        logger.setLevel(logging.INFO)
        logger_file_handler = logging.handlers.RotatingFileHandler(
            f"logs/{datetime.now().strftime('%d-%m-%Y')}.log",
            maxBytes=1024 * 1024,
            backupCount=1,
            encoding="utf8",
        )
        formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
        logger_file_handler.setFormatter(formatter)
        logger.addHandler(logger_file_handler)
        
        # Get API token
        TOKEN = get_api_token()

        # Get the API response
        url = "https://api.waqi.info/search/?token=" + TOKEN + "&keyword=" + station
        response = requests.get(url)
        if response.status_code == 200:
            res = response.json()
            result = []
            if station.lower() in (res["data"][0]['station']['name']).lower():
                result.append(res["data"][0]['aqi'])
                result.append(res["data"][0]['station']['name'])
                result.append(pd.to_datetime(res["data"][0]['time']['stime']))
            logger.info(f"station=> {station}, result => {result}")
            print(f"station=> {station}, result => {result}")
            
            # Write to the file only when (station, time) is not already existing in the file.

            new_timestamp = (res["data"][0]['time']['stime'])
            csv_file_path = output_file + '.csv'

            # Check if the new timestamp is already present
            with open(csv_file_path, 'r') as csv_file:
                csv_reader = csv.reader(csv_file)
                # Assuming the timestamp is in the 3rd column
                existing_timestamps = [row[2] for row in csv_reader]

            if new_timestamp not in existing_timestamps:
                with open(csv_file_path, 'a', newline='') as csv_file:
                    csv_writer = csv.writer(csv_file)
                    csv_writer.writerow(result)
                print(f'The data has been written to {csv_file_path} with Timestamp: {new_timestamp}')
                logger.info(f'The data has been written to {csv_file_path} with Timestamp: {new_timestamp}')
            else:
                print(f'Timestamp {new_timestamp} already present in {csv_file_path}, not appending.')
                logger.info(f'Timestamp {new_timestamp} already present in {csv_file_path}, not appending.')
        else:
            print(f"Error: {response.status_code} - {response.text}")
            logger.info(f"Error: {response.status_code} - {response.text}")
    except Exception as e:
        
        logging.info("1")
        logging.debug("2")
#         print(f"Exception {type(e).__name__} has occured for station=> {station}")
        logger.info(f"Exception {type(e).__name__} has occured for station=> {station}")
        
def setLogger():
    logger = logging.getLogger(__name__)
    logger.setLevel(logging.INFO)
    logger_file_handler = logging.handlers.RotatingFileHandler(
        f"logs/{datetime.now().strftime('%d-%m-%Y')}.log",
        maxBytes=1024 * 1024,
        backupCount=1,
        encoding="utf8",
    )
    formatter = logging.Formatter("%(asctime)s - %(name)s - %(levelname)s - %(message)s")
    logger_file_handler.setFormatter(formatter)
    logger.addHandler(logger_file_handler)
    return logger


In [387]:
if __name__ == "__main__":
    stations = [(NISE,  NISE_OUTPUT), (SECTOR_51, SECTOR_51_OUTPUT), (TERI_GRAM, TERI_GRAM_OUTPUT), (VIKAS_SADAN, VIKAS_SADAN_OUTPUT)]
    for station, station_location in stations:
        setData(station,  station_location)

KeyError
Enter your API token: 7c0e1c5a796cf1a14edf4bf1462a99e9b37d8bdf


AttributeError: 'float' object has no attribute 'astimezone'

In [374]:
def writeData(station_location):
    df_api = pd.read_csv(station_location + ".csv")
    df_api['Time'] = pd.to_datetime(df_api['Time'])
    df_api.set_index('Time', inplace=True)
    df_api.resample('D').mean()

In [376]:
if __name__ == "__main__":

    # Logging
    logger = setLogger()
    
    # Get API token
    TOKEN = get_api_token()

    for station, station_location in stations:
        setData(station,  station_location, logger, TOKEN)

    # If the day changes, append it to original data

#     if datetime.now().hour == 1:     # It means 1 AM
#         for station, station_location in stations:
#             writeData(station_location)

KeyError
Enter your API token: 7c0e1c5a796cf1a14edf4bf1462a99e9b37d8bdf
Exception AttributeError has occured for station=> nise gwal


AttributeError: 'float' object has no attribute 'astimezone'

In [377]:
datetime.now().hour

8

In [378]:
datetime.now().astimezone().timetuple()

time.struct_time(tm_year=2023, tm_mon=12, tm_mday=1, tm_hour=8, tm_min=2, tm_sec=44, tm_wday=4, tm_yday=335, tm_isdst=-1)

In [379]:
sector_51_api = pd.read_csv("../../data/cron_job_data/sector_51_cron_output.csv")
sector_51_api

,AQI,Station_name,Time
0,183,"Sector-51, Gurugram, India",2023-11-28 20:00:00
1,203,"Sector-51, Gurugram, India",2023-11-28 21:00:00
2,200,"Sector-51, Gurugram, India",2023-11-28 22:00:00
3,200,"Sector-51, Gurugram, India",2023-11-28 23:00:00
4,197,"Sector-51, Gurugram, India",2023-11-29 00:00:00
5,195,"Sector-51, Gurugram, India",2023-11-29 01:00:00
6,197,"Sector-51, Gurugram, India",2023-11-29 02:00:00
7,200,"Sector-51, Gurugram, India",2023-11-29 03:00:00
8,200,"Sector-51, Gurugram, India",2023-11-29 04:00:00
9,200,"Sector-51, Gurugram, India",2023-11-29 05:00:00


In [380]:
sector_51_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   AQI           51 non-null     int64 
 1   Station_name  51 non-null     object
 2   Time          51 non-null     object
dtypes: int64(1), object(2)
memory usage: 1.3+ KB


In [381]:
sector_51_api['Time'] = pd.to_datetime(sector_51_api['Time'])
sector_51_api.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 51 entries, 0 to 50
Data columns (total 3 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   AQI           51 non-null     int64         
 1   Station_name  51 non-null     object        
 2   Time          51 non-null     datetime64[ns]
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 1.3+ KB


In [382]:
sector_51_api.set_index('Time', inplace=True)


In [388]:
sector_51_api = sector_51_api.resample('D').mean()
sector_51_api

,AQI
Time,
2023-11-28,196.500000
2023-11-29,227.157895
2023-11-30,253.238095
2023-12-01,185.142857


In [384]:
# Get today's date
today = datetime.now().date()

# Calculate yesterday's date
yesterday = today - timedelta(days=1)
yesterday

datetime.date(2023, 11, 30)

In [395]:
sector_51_api[yesterday:yesterday]

,AQI
Time,
2023-11-30,253.238095
